In [5]:
import configparser
import io
import os
from collections import defaultdict
import numpy as np
import tensorflow
import keras
import h5py
print(tensorflow.__version__, keras.__version__)
#install tensorflow above version 1.3 
from keras import backend as K
from keras.layers import (Conv2D, GlobalAveragePooling2D, Input, Lambda,
                          MaxPooling2D)
from keras.layers import LeakyReLU
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Flatten, Dense, Activation, Reshape
from keras.models import load_model
from keras.models import Model
from keras.models import Sequential
from keras.regularizers import l2
from keras.utils.vis_utils import plot_model as plot

import cv2
import matplotlib.pyplot as plt


1.5.0 2.1.5


In [6]:
def unique_config_sections(config_file):
    """Convert all config sections to have unique names.
    Adds unique suffixes to config sections for compability with configparser.
    """
    section_counters = defaultdict(int)
    output_stream = io.StringIO()
    fin=config_file
    newfilename='newconfig_file.cfg'
    output_stream= open('newconfig_file.cfg','w')
    for line in fin:
        #print(line)
        if line.startswith('['):
            section = line.strip().strip('[]')
            _section = section + '_' + str(section_counters[section])
            section_counters[section] += 1
            line = line.replace(section, _section)
        output_stream.write(line)
    output_stream.seek(0)
    return newfilename

In [7]:
#m=createKerasmodel('yolo-small.cfg')
##wrong model is being created by this method check this method later just load from existing model
m=load_model('model_keras.h5')
print(m.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 416, 416, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 416, 416, 16)      432       
_________________________________________________________________
batch_normalization_1 (Batch (None, 416, 416, 16)      64        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 416, 416, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 208, 208, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 208, 208, 32)      4608      
_________________________________________________________________
batch_normalization_2 (Batch (None, 208, 208, 32)      128       
__________

//anaconda/envs/cs_sop/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [8]:
class BoundBox:
    def __init__(self,classes):
        self.x, self.y = float(), float()
        self.w, self.h = float(), float()
        self.c = float()
        self.class_num=classes
        self.probs=np.zeros((classes,))
  

In [9]:
def overlap(x1,w1,x2,w2):
    l1 = x1 - w1 / 2.;
    l2 = x2 - w2 / 2.;
    left = max(l1, l2)
    r1 = x1 + w1 / 2.;
    r2 = x2 + w2 / 2.;
    right = min(r1, r2)
    return right - left;

In [10]:
def box_intersection(ax,ay,aw,ah,bx,by,bw,bh):
    w = overlap(ax, aw, bx, bw);
    h = overlap(ay, ah, by, bh);
    if w < 0 or h < 0: return 0;
    area = w * h;
    return area;



In [11]:
def box_union(ax,ay,aw,ah,bx,by,bw,bh):
    i = box_intersection(ax,ay,aw,ah,bx,by,bw,bh);
    u = aw * ah + bw * bh - i;
    return u;



In [12]:
def box_iou(ax,ay,aw,ah,bx,by,bw,bh):
    return box_intersection(ax,ay,aw,ah,bx,by,bw,bh) / box_union(ax,ay,aw,ah,bx,by,bw,bh);



In [13]:
def NMS(final_probs , final_bbox):
    '''
    Function to perform non max suppresion of bounding boxes.
    Eliminates all those boxes having iou>=0.4
    '''
    boxes=list()
    indices=set()
    pred_length = final_bbox.shape[0]
    class_length = final_probs.shape[1]
    for class_loop in range(class_length):
        for index in range(pred_length):
            if final_probs[index,class_loop] == 0: 
                continue
            for index2 in range(index+1,pred_length):
                if final_probs[index2,class_loop] == 0: 
                    continue
                if index==index2 : continue
                if box_iou(final_bbox[index,0],final_bbox[index,1],final_bbox[index,2],final_bbox[index,3],final_bbox[index2,0],final_bbox[index2,1],final_bbox[index2,2],final_bbox[index2,3]) >= 0.4:
                    if final_probs[index2,class_loop] > final_probs[index, class_loop] :
                        final_probs[index, class_loop] =0
                        break
                    final_probs[index2,class_loop]=0
            
            if index not in indices:
                bb=BoundBox(class_length)
                bb.x = final_bbox[index, 0]
                bb.y = final_bbox[index, 1]
                bb.w = final_bbox[index, 2]
                bb.h = final_bbox[index, 3]
              #  bb.c = final_bbox[index, 4]
                bb.probs = np.asarray(final_probs[index,:])
                boxes.append(bb)
                indices.add(index)
    return boxes


In [14]:
def expit(x):
    #as given in the paper 
    return 1. / (1. + np.exp(-x))


In [15]:
def box_constructorv2(net_out_in,threshold):
   
    #hard- coded for yolov2-voc from config file
    arr_max=0
    sum=0
    anchors = np.asarray([1.08,1.19,  3.42,4.41,  6.63,11.38,  9.42,5.11,  16.62,10.52])
    boxes = list()
    #wrong mostly
    H, W, _ = [13,13,125]
    #C = meta['classes']
    C=20
    B=5
    #B = meta['num']
    
    #net_out_in.shape[2]=125
    net_out = net_out_in.reshape([H, W, B, 125//B])
    Classes = net_out[:, :, :, 5:]
    Bbox_pred =  net_out[:, :, :, :5]
    probs = np.zeros((H, W, B, C), dtype=np.float32)
    
    for row in range(H):
        for col in range(W):
            for box_loop in range(B):
                arr_max=0
                sum=0
                Bbox_pred[row, col, box_loop, 4] = expit(Bbox_pred[row, col, box_loop, 4])
                Bbox_pred[row, col, box_loop, 0] = (col + expit(Bbox_pred[row, col, box_loop, 0])) / W
                Bbox_pred[row, col, box_loop, 1] = (row + expit(Bbox_pred[row, col, box_loop, 1])) / H
                Bbox_pred[row, col, box_loop, 2] = np.exp(Bbox_pred[row, col, box_loop, 2]) * anchors[2 * box_loop + 0] / W
                Bbox_pred[row, col, box_loop, 3] = np.exp(Bbox_pred[row, col, box_loop, 3]) * anchors[2 * box_loop + 1] / H
                #SOFTMAX BLOCK, no more pointer juggling
                for class_loop in range(C):
                    arr_max=max(arr_max,Classes[row,col,box_loop,class_loop])
                
                for class_loop in range(C):
                    Classes[row,col,box_loop,class_loop]=np.exp(Classes[row,col,box_loop,class_loop]-arr_max)
                    sum+=Classes[row,col,box_loop,class_loop]
                
                for class_loop in range(C):
                    tempc = Classes[row, col, box_loop, class_loop] * Bbox_pred[row, col, box_loop, 4]/sum                    
                    if(tempc > threshold):
                        probs[row, col, box_loop, class_loop] = tempc
    
    
    #NMS- details of output vector postprocessing in paper-refer figure                   
    return NMS(np.ascontiguousarray(probs).reshape(H*W*B,C), np.ascontiguousarray(Bbox_pred).reshape(H*B*W,5))

In [16]:
def yolo_box_constructor(net_out,threshold):
    con_file=open('yolov2-tiny-voc.cfg','r')
    print('the net out received is')
    print(net_out)
    unique_config_file=unique_config_sections(con_file)
    cfg_parser = configparser.ConfigParser()
    cfg_parser.read(unique_config_file)
    C=int (cfg_parser['region_0']['classes'])
    
    B=int (cfg_parser['region_0']['num'])
    S=int(cfg_parser['region_0']['side'])
    print(C,B,S)
    sqrt=int (cfg_parser['detection_0']['sqrt'])+1
    boxes=[]
    count=0
    SS=S*S
    prob_size=SS*C
    conf_size=SS*B
    probs = np.ascontiguousarray(net_out[0 : prob_size]).reshape([SS,C])
    print(probs)
    confs =  np.ascontiguousarray(net_out[prob_size : (prob_size + conf_size)]).reshape([SS,B])
    coords =  np.ascontiguousarray(net_out[(prob_size + conf_size) : ]).reshape([SS, B, 4])
    final_probs = np.zeros([SS,B,C],dtype=np.float32)
    for grid in range(SS):
        for b in range(B):
            coords[grid, b, 0] = (coords[grid, b, 0] + grid %  S) / S
            coords[grid, b, 1] = (coords[grid, b, 1] + grid // S) / S
            coords[grid, b, 2] =  coords[grid, b, 2] ** sqrt
            coords[grid, b, 3] =  coords[grid, b, 3] ** sqrt
            for class_loop in range(C):
                #did not multiply by confidence measure
                probs[grid, class_loop] = probs[grid, class_loop]*confs[grid, b]
                print("PROBS",probs[grid,class_loop])
                if(probs[grid,class_loop] > threshold ):
                    final_probs[grid, b, class_loop] = probs[grid, class_loop]*confs[grid, b]
                    count+=1
    
    #changed to 5 
    print('length of original no of boxes=',count)
    return NMS(np.ascontiguousarray(final_probs).reshape(SS*B, C) , np.ascontiguousarray(coords).reshape(SS*B, 4))

In [17]:
#preprocessing the image
imagePath = 'test12.jpg'
image = cv2.imread(imagePath)
print (image.shape)
#cv2.imshow('image',image)
cv2.imwrite('imagechange.jpg',image)
#f,(ax1,ax2) = plt.subplots(1,2,figsize=(16,6))
#ax1.imshow(image)
resized = cv2.resize(image,(416,416))
print(resized.shape,'shape of resized')
cv2.imwrite('resizedimage.jpg',resized)
#resized=resized/255.0
#check if we need to normalize the image
print(resized.shape)

imarray=resized/255.0

#imarray=np.transpose(imarray)
#imarray=imarray.reshape(416,416,3,-1)
imarray=np.expand_dims(imarray,axis=0)

print(imarray.shape)

#print(imtrans.shape)
out=m.predict(imarray)

print(out.shape)

(620, 980, 3)
(416, 416, 3) shape of resized
(416, 416, 3)
(1, 416, 416, 3)
(1, 13, 13, 125)


In [18]:
print(out[0].shape)

(13, 13, 125)


In [19]:
def process_box(b,h,w,threshold):
    max_indx=np.argmax(b.probs)
    max_prob=b.probs[max_indx]
    labels=['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable','dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']
    label=labels[max_indx]
    if max_prob > threshold:
        left  = int ((b.x - b.w/2.) * w)
        right = int ((b.x + b.w/2.) * w)
        top   = int ((b.y - b.h/2.) * h)
        bot   = int ((b.y + b.h/2.) * h)
        if left  < 0    :  left = 0
        if right > w - 1: right = w - 1
        if top   < 0    :   top = 0
        if bot   > h - 1:   bot = h - 1
        mess = '{}'.format(label)
        return (left, right, top, bot, mess, max_indx, max_prob)
    return None

In [20]:
def post_processing_out(net_out,original_im,threshold):
    boxes=box_constructorv2(net_out,threshold)
    print(len(boxes))
    imgcv=cv2.imread(original_im)
    img_name='outimage.jpg'
    
    colors=['red','red','red','red','red','blue','blue''blue','blue','blue','green','green','green','green','green','yellow','yellow','yellow','yellow','yellow']
    h,w,_=imgcv.shape
    print(imgcv.shape)
    for b in boxes:
        print('entering boxes')
        boxResults=process_box(b,h,w,threshold)
        if boxResults is None:
            print('Box results is none')
            continue
        left,right,top,bot,mess,max_indx,confidence=boxResults
        
        thick=int((h+w)//300)
        cv2.rectangle(imgcv,(left, top), (right, bot),(0,255,0), thick)
        print(mess)
        cv2.putText(imgcv, mess, (left, top - 12),0, 1e-3 * h, (0,255,0),thick // 3)
    cv2.imwrite(img_name,imgcv)

In [21]:
post_processing_out(out[0],'test12.jpg',0.4)

1
(620, 980, 3)
entering boxes
dog
